In [1]:
from google.colab import drive
drive.mount("/content/drive")
import os
os.chdir("/content/drive/MyDrive/Test")
os.listdir()

Mounted at /content/drive


['Bitcoin.csv',
 'GBP_TRY.csv',
 'EUR_USD.csv',
 'Exchange_Calculator.ipynb',
 'Usd_Try.ipynb',
 'GBP_Try.ipynb',
 'Euro_usd.ipynb',
 'XAU_USD.csv',
 'BT_Exchange.ipynb',
 'Gold_USD.ipynb',
 'USD_TRY.csv']

In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [3]:
data_usd_tr = pd.read_csv("USD_TRY.csv")
data_usd_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1295 entries, 0 to 1294
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tarih   1295 non-null   object
 1   Şimdi   1295 non-null   object
 2   Açılış  1295 non-null   object
 3   Yüksek  1295 non-null   object
 4   Düşük   1295 non-null   object
 5   Hac.    672 non-null    object
 6   Fark %  1295 non-null   object
dtypes: object(7)
memory usage: 70.9+ KB


In [4]:
df = data_usd_tr.drop(['Açılış','Yüksek','Düşük','Hac.','Fark %'], axis=1)

In [5]:
df.head(10)

,Tarih,Şimdi
0,16.12.2022,"18,6343"
1,15.12.2022,"18,6368"
2,14.12.2022,"18,6232"
3,13.12.2022,"18,5750"
4,12.12.2022,"18,6325"
5,09.12.2022,"18,6503"
6,08.12.2022,"18,6324"
7,07.12.2022,"18,6355"
8,06.12.2022,"18,6240"
9,05.12.2022,"18,6227"


In [6]:
# We need to change from "," to "." because it will be a float.
df['Şimdi'] = df['Şimdi'].apply(lambda x: float(x.split()[0].replace(',', '.')))
df

,Tarih,Şimdi
0,16.12.2022,18.6343
1,15.12.2022,18.6368
2,14.12.2022,18.6232
3,13.12.2022,18.5750
4,12.12.2022,18.6325
...,...,...
1290,05.01.2018,3.7367
1291,04.01.2018,3.7480
1292,03.01.2018,3.7812
1293,02.01.2018,3.7667


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1295 entries, 0 to 1294
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Tarih   1295 non-null   object 
 1   Şimdi   1295 non-null   float64
dtypes: float64(1), object(1)
memory usage: 20.4+ KB


In [8]:
# univariate data preparation
from numpy import array
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			 break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
raw_seq = df['Şimdi']
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[18.6343 18.6368 18.6232 18.575  18.6325] 18.6503
[18.6368 18.6232 18.575  18.6325 18.6503] 18.6324
[18.6232 18.575  18.6325 18.6503 18.6324] 18.6355
[18.575  18.6325 18.6503 18.6324 18.6355] 18.624
[18.6325 18.6503 18.6324 18.6355 18.624 ] 18.6227
[18.6503 18.6324 18.6355 18.624  18.6227] 18.6303
[18.6324 18.6355 18.624  18.6227 18.6303] 18.6313
[18.6355 18.624  18.6227 18.6303 18.6313] 18.5895
[18.624  18.6227 18.6303 18.6313 18.5895] 18.6374
[18.6227 18.6303 18.6313 18.5895 18.6374] 18.6166
[18.6303 18.6313 18.5895 18.6374 18.6166] 18.5851
[18.6313 18.5895 18.6374 18.6166 18.5851] 18.5986
[18.5895 18.6374 18.6166 18.5851 18.5986] 18.6062
[18.6374 18.6166 18.5851 18.5986 18.6062] 18.6081
[18.6166 18.5851 18.5986 18.6062 18.6081] 18.6118
[18.5851 18.5986 18.6062 18.6081 18.6118] 18.5889
[18.5986 18.6062 18.6081 18.6118 18.5889] 18.5599
[18.6062 18.6081 18.6118 18.5889 18.5599] 18.592
[18.6081 18.6118 18.5889 18.5599 18.592 ] 18.602
[18.6118 18.5889 18.5599 18.592  18.602 ] 18.5846
[18

In [9]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[df['Şimdi'][0],df['Şimdi'][1],df['Şimdi'][2],df['Şimdi'][3],df['Şimdi'][4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

Epoch 1/200
41/41 [==============================] - 4s 55ms/step - loss: 16.5942
Epoch 2/200
41/41 [==============================] - 3s 61ms/step - loss: 0.5388
Epoch 3/200
41/41 [==============================] - 3s 65ms/step - loss: 0.1029
Epoch 4/200
41/41 [==============================] - 3s 65ms/step - loss: 0.0826
Epoch 5/200
41/41 [==============================] - 3s 66ms/step - loss: 0.0756
Epoch 6/200
41/41 [==============================] - 2s 60ms/step - loss: 0.0799
Epoch 7/200
41/41 [==============================] - 1s 33ms/step - loss: 0.0726
Epoch 8/200
41/41 [==============================] - 1s 31ms/step - loss: 0.0693
Epoch 9/200
41/41 [==============================] - 1s 31ms/step - loss: 0.0677
Epoch 10/200
41/41 [==============================] - 1s 32ms/step - loss: 0.0674
Epoch 11/200
41/41 [==============================] - 1s 32ms/step - loss: 0.0675
Epoch 12/200
41/41 [==============================] - 1s 32ms/step - loss: 0.0671
Epoch 13/200
41/41 [====

In [23]:
print("Prediction: ",round(yhat[0][0],2), "₺ USD/TL") # onluk olarak yukarı yuvarlandı.

Prediction:  18.88 ₺ USD/TL
